In [11]:
import json
import os
from pathlib import Path
import pandas as pd

from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from pgvector.sqlalchemy import Vector
from sqlalchemy import Index, create_engine, select, text
from sqlalchemy.orm import Session


In [12]:

# Connect to the database based on environment variables
load_dotenv(".env", override=True)
POSTGRES_HOST = os.environ["POSTGRES_HOST"]
POSTGRES_USERNAME = os.environ["POSTGRES_USERNAME"]
POSTGRES_DATABASE = os.environ["POSTGRES_DATABASE"]

if POSTGRES_HOST.endswith(".database.azure.com"):
    print("Authenticating to Azure Database for PostgreSQL using Azure Identity...")
    azure_credential = DefaultAzureCredential()
    token = azure_credential.get_token("https://ossrdbms-aad.database.windows.net/.default")
    POSTGRES_PASSWORD = token.token
else:
    POSTGRES_PASSWORD = os.environ["POSTGRES_PASSWORD"]

DATABASE_URI = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{POSTGRES_DATABASE}"
# Specify SSL mode if needed
if POSTGRES_SSL := os.environ.get("POSTGRES_SSL"):
    DATABASE_URI += f"?sslmode={POSTGRES_SSL}"

engine = create_engine(DATABASE_URI, echo=False)

Authenticating to Azure Database for PostgreSQL using Azure Identity...


In [16]:
# Run query and print results
query="select * from recommend_recipe_by_description('chicken and peanut',5)"
with Session(engine) as session:
    most_similars = session.execute(text(query)) #.scalars()
    print(f"Most similar recipes to 'chicken and peanut':")
    print(f"--------------------------------------------------")
    for Recipe in most_similars:
        print(f"INPUT: {Recipe[0]}: \nOUTPUT: {Recipe[1]} SCORE: ({Recipe[2]})")
        print(f"--------------------------------------------------")

Most similar recipes to 'chicken and peanut':
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Apricot Glazed Chicken SCORE: (Total Fat 2g 2%, Saturated Fat 1g 3%, Cholesterol 83mg 28%, Sodium 227mg 10%, Total Carbohydrate 35g 13%, Total Sugars 31g, Protein 34g, Vitamin C 2mg 9%, Calcium 16mg 1%, Iron 1mg 6%, Potassium 367mg 8%)
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Chicken, Avocado and Mango Salad SCORE: (Total Fat 16g 21%, Saturated Fat 4g 18%, Cholesterol 55mg 18%, Sodium 698mg 30%, Total Carbohydrate 20g 7%, Dietary Fiber 5g 19%, Total Sugars 12g, Protein 19g, Vitamin C 31mg 153%, Calcium 37mg 3%, Iron 2mg 11%, Potassium 526mg 11%)
--------------------------------------------------
INPUT: chicken and peanut: 
OUTPUT: Green Papaya Salad (Som Tam Malakor) SCORE: (Total Fat 10g 13%, Saturated Fat 1g 7%, Cholesterol 5mg 2%, Sodium 1126mg 49%, Total Carbohydrate 37g 14%, Dietary Fiber 7g 26%, Total Su